<a href="https://colab.research.google.com/github/SEAS-CVN/SEAS-2025/blob/main/P1_Flood.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ứng Dụng Trí Tuệ Nhân Tạo Trong Dự Báo Lũ Lụt Tại Miền Trung Việt Nam

## Bối cảnh:
Miền Trung Việt Nam, với mạng lưới sông ngòi và kênh rạch dày đặc, rất dễ bị ảnh hưởng bởi các biến động khí hậu và thường xuyên hứng chịu những trận lũ lụt nghiêm trọng. Những trận lũ này, ngày càng trầm trọng do biến đổi khí hậu, nạn phá rừng và đô thị hóa, gây ra những thiệt hại lớn về nông nghiệp, cơ sở hạ tầng và tính mạng con người. Là một trong những quốc gia chịu ảnh hưởng nặng nề nhất bởi rủi ro khí hậu và nguy cơ lũ lụt trên toàn cầu, Việt Nam đang rất cần các chiến lược dự báo và giảm thiểu lũ hiệu quả hơn.

## Mục tiêu:
Mục tiêu của dự án này là phát triển một mô hình học máy có khả năng dự đoán mực nước ở một sông thuộc tỉnh Quảng Trị để có thể biết trước được

## Chi tiết:
Trong dự án này, chúng ta sử dụng bộ dữ liệu CAMELS - một tập dữ liệu thủy văn quy mô lớn, bao gồm thuộc tính lưu vực (như độ cao, thảm phủ, đất...) và dữ liệu chuỗi thời gian (như lượng mưa, nhiệt độ, dòng chảy) của gần 700 lưu vực tại Hoa Kỳ.
Do CAMELS không có dữ liệu tại Việt Nam, nhóm sẽ sử dụng các lưu vực có điều kiện tương đồng với sông Long Đại (Quảng Trị) để huấn luyện mô hình học máy.
Sau khi mô hình được huấn luyện, ta sẽ áp dụng mô hình này cho lưu vực sông Long Đại bằng cách cung cấp các thông tin thuộc tính tương ứng (dữ liệu địa phương). Mục tiêu là dự báo mực nước tại sông Long Đại, từ đó hỗ trợ cảnh báo sớm và giảm thiểu rủi ro do lũ.



## Khám phá dữ liệu

Kết nối Google Drive để lưu trữ và truy cập dữ liệu cần thiết cho dự án. Thực hiện lệnh sau để gắn Google Drive vào môi trường làm việc:

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
#Đường dẫn tới dữ liệu
data_path = '/content/drive/MyDrive/data'

### Tổng quan dữ liệu
#### Vì sao chọn CAMELS?
- CAMELS (Catchment Attributes and Meteorology for Large-sample Studies) là bộ dữ liệu chuẩn được sử dụng rộng rãi trong các nghiên cứu về thủy văn và mô hình học máy.
- So với các bộ phức tạp như Caravan, CAMELS có:
  + Ít lưu vực hơn (gần 700).
  + Quy mô vừa phải, cấu trúc đơn giản, dễ xử lý.
  + Chia thành 2 phần rõ ràng: thuộc tính tĩnh (attributes) và dữ liệu chuỗi thời gian (time series).
  + Được tổ chức theo các nhóm đặc trưng (địa hình, khí hậu, đất, lớp phủ đất,...).
  + Dễ dàng áp dụng trong các bài toán học máy như phân cụm, PCA, dự báo dòng chảy,...

#### Cấu trúc bộ dữ liệu
- Dữ liệu thuộc tính lưu vực (basin attributes) - tĩnh, không thay đổi theo thời gian. Lưu trong `basin_set_full_res.csv`. Gồm các nhóm thuộc tính như:
  + Địa hình (Độ cao, độ dốc, diện tích lưu vực):	elev_mean, slope_mean, area_gages2
  + Khí hậu	(Mưa, bốc hơi, chỉ số khô hạn,...):	p_mean, pet_mean, aridity, p_seasonality
  + Lớp phủ đất (Rừng, diện tích lá, loại lớp phủ đất):	frac_forest, lai_max, dom_land_cover
  + Đất	(Độ sâu đất, thành phần cát/bùn/sét, độ thấm):	soil_depth_statsgo, sand_frac, clay_frac, soil_porosity
  + Địa chất (Loại đá, độ thấm của nền đất):	geol_class_1st, geol_porosity, geol_permeability
  + Dòng chảy	(Lưu lượng, chỉ số dòng chảy, baseflow): q_mean, runoff_ratio, baseflow_index
- Dữ liệu chuỗi thời gian (time series) - theo ngày. Lưu trong `basin_timeseries_v1p2_metForcing_obsFlow`. Gồm 2 loại chính:
  + Dữ liệu khí tượng đầu vào (meteorological forcings):
    + Nguồn: daymet, maurer, nldas, prism,...
    + Các trường: precipitation, temperature, solar_radiation, humidity, wind...
  + Dòng chảy quan trắc (observed streamflow): Lưu lượng thực đo tại cửa lưu vực.

Cài đặt các thư viện Python cần thiết cho việc xử lý dữ liệu, trực quan hóa và phân tích. Chạy các lệnh sau để cài đặt:
- [cartopy](https://pypi.org/project/Cartopy/): Thư viện giúp việc vẽ bản đồ phục vụ phân tích và trực quan hóa dữ liệu trở nên dễ dàng.
- [folium](https://pypi.org/project/folium/): Thư viện bản đồ tương tác dựa trên Leaflet.js, dùng để hiển thị bản đồ trực tuyến (ví dụ: bản đồ nhiệt, đánh dấu vị trí,...).
- [python-ternary](https://pypi.org/project/python-ternary/): Thư viện vẽ biểu đồ tam giác (ternary plots), dùng để thể hiện dữ liệu có ba thành phần tổng bằng 1.

In [ ]:
!pip install cartopy
!pip install folium
!pip install python-ternary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 106.5 MB/s eta 0:00:00


In [ ]:
#Include các thư viện cần thiết
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np
import glob
from datetime import datetime, timedelta
import logging

## Preprocessing data


### Đọc và phân tích dữ liệu


In [ ]:
%cd "/content/drive/MyDrive/data"

# Define recommended columns for Vietnam analysis (Chọn các đặc trưng tiêu biểu, sử dụng để lọc ra các gauges phù hợp với khí hậu và sông ngòi Việt Nam)
recommended_cols = {
    "name": ["gauge_id", "gauge_name"],
    "clim": ["gauge_id", "p_mean", "pet_mean", "aridity", "p_seasonality"],
    "topo": ["gauge_id", "gauge_lon", "gauge_lat", "elev_mean", "slope_mean", "area_gages2"],
    "soil": ["gauge_id", "soil_depth_statsgo", "sand_frac", "silt_frac", "clay_frac",
             "soil_porosity", "soil_conductivity"],
    "geol": ["gauge_id", "geol_1st_class", "geol_porostiy", "geol_permeability"],
    "vege": ["gauge_id", "frac_forest", "dom_land_cover", "lai_max", "gvf_max", "gvf_diff"]
}


# Load data
name_df = pd.read_csv('camels_name.txt', sep=';')[recommended_cols["name"]]
clim_df = pd.read_csv('camels_clim.txt', sep=';')[recommended_cols["clim"]]
topo_df = pd.read_csv('camels_topo.txt', sep=';')[recommended_cols["topo"]]
soil_df = pd.read_csv('camels_soil.txt', sep=';')[recommended_cols["soil"]]
geol_df = pd.read_csv('camels_geol.txt', sep=';')[recommended_cols["geol"]]
vege_df = pd.read_csv('camels_vege.txt', sep=';')[recommended_cols["vege"]]

print(f"Name data shape: {name_df.shape}")
print(f"Climate data shape: {clim_df.shape}")
print(f"Topography data shape: {topo_df.shape}")
print(f"Soil data shape: {soil_df.shape}")
print(f"Geological data shape: {geol_df.shape}")
print(f"Vegetation data shape: {vege_df.shape}")

# Merge all data
df = (name_df.merge(clim_df, on='gauge_id')
             .merge(topo_df, on='gauge_id')
             .merge(soil_df, on='gauge_id')
             .merge(geol_df, on='gauge_id')
             .merge(vege_df, on='gauge_id'))
print(f"Merged data shape: {df.shape}")

print("\nCLIMATE ATTRIBUTES:")
for col in recommended_cols["clim"][1:]:  # Skip gauge_id
    if col in df.columns:
        stats = df[col].describe()
        print(f"  {col}: min={stats['min']:.3f}, mean={stats['mean']:.3f}, max={stats['max']:.3f}")

print("\nTOPOGRAPHY ATTRIBUTES:")
for col in recommended_cols["topo"][1:]:
    if col in df.columns:
        stats = df[col].describe()
        print(f"  {col}: min={stats['min']:.3f}, mean={stats['mean']:.3f}, max={stats['max']:.3f}")

print("\nSOIL ATTRIBUTES:")
for col in recommended_cols["soil"][1:]:
    if col in df.columns:
        stats = df[col].describe()
        print(f"  {col}: min={stats['min']:.3f}, mean={stats['mean']:.3f}, max={stats['max']:.3f}")

print("\nVEGETATION ATTRIBUTES:")
for col in recommended_cols["vege"][1:]:
    if col in df.columns and pd.api.types.is_numeric_dtype(df[col]):
        stats = df[col].describe()
        print(f"  {col}: min={stats['min']:.3f}, mean={stats['mean']:.3f}, max={stats['max']:.3f}")

print("\nGEOLOGICAL ATTRIBUTES:")
for col in recommended_cols["geol"][1:]:
    if col in df.columns:
        if pd.api.types.is_numeric_dtype(df[col]):
            stats = df[col].describe()
            print(f"  {col}: min={stats['min']:.3f}, mean={stats['mean']:.3f}, max={stats['max']:.3f}")
        else:
            unique_vals = df[col].value_counts().head(5)
            print(f"  {col}: Top 5 types = {list(unique_vals.index)}")


/content/drive/MyDrive/data
Name data shape: (671, 2)
Climate data shape: (671, 5)
Topography data shape: (671, 6)
Soil data shape: (671, 7)
Geological data shape: (671, 4)
Vegetation data shape: (671, 6)
Merged data shape: (671, 25)

CLIMATE ATTRIBUTES:
  p_mean: min=0.645, mean=3.258, max=8.937
  pet_mean: min=1.899, mean=2.787, max=4.744
  aridity: min=0.220, mean=1.057, max=5.208
  p_seasonality: min=-1.435, mean=-0.041, max=0.922

TOPOGRAPHY ATTRIBUTES:
  gauge_lon: min=-124.393, mean=-95.790, max=-67.935
  gauge_lat: min=27.052, mean=39.245, max=48.823
  elev_mean: min=10.210, mean=759.422, max=3571.180
  slope_mean: min=0.822, mean=46.195, max=255.688
  area_gages2: min=4.030, mean=792.618, max=25791.040

SOIL ATTRIBUTES:
  soil_depth_statsgo: min=0.400, mean=1.293, max=1.500
  sand_frac: min=8.184, mean=36.468, max=91.976
  silt_frac: min=2.985, mean=33.859, max=67.775
  clay_frac: min=1.846, mean=19.886, max=50.354
  soil_porosity: min=0.373, mean=0.443, max=0.680
  soil_condu

Để biết các vị trí của các máy đo trong dữ liệu của CAMELS, các bạn có thể dùng các code dưới đây:

In [ ]:
import folium

def plot_gauge_locations(df, lat_col='gauge_lat', lon_col='gauge_lon', popup_col='gauge_name', zoom_start=4):
    """
    Vẽ các vị trí trạm đo trên bản đồ tương tác sử dụng folium.

    Tham số:
    - df: DataFrame chứa dữ liệu vị trí trạm đo.
    - lat_col: Tên cột chứa giá trị vĩ độ.
    - lon_col: Tên cột chứa giá trị kinh độ.
    - popup_col: Cột sẽ được sử dụng làm chú thích khi bấm vào điểm (popup). Nếu là None, sẽ không hiển thị popup.
    - zoom_start: Mức độ thu phóng ban đầu của bản đồ.

    Trả về:
    - Đối tượng folium.Map.
    """
    if lat_col not in df.columns or lon_col not in df.columns:
        raise ValueError("Không tìm thấy cột vĩ độ hoặc kinh độ trong DataFrame.")

    lat_center = df[lat_col].mean()
    lon_center = df[lon_col].mean()

    # Khởi tạo bản đồ
    m = folium.Map(location=[lat_center, lon_center], zoom_start=zoom_start)

    # Thêm từng trạm đo vào bản đồ dưới dạng CircleMarker
    for _, row in df.iterrows():
        popup_text = f"{popup_col}: {row[popup_col]}" if popup_col and popup_col in row else None
        folium.CircleMarker(
            location=[row[lat_col], row[lon_col]],
            radius=5,
            color='red',
            fill=True,
            fill_opacity=0.7,
            popup=popup_text
        ).add_to(m)

    return m

In [ ]:
plot_gauge_locations(df)

## Đặc điểm tự nhiên của Việt Nam

### KHÍ HẬU:
- **Lượng mưa trung bình (`p_mean`):** **5.5 mm/ngày** (tương đương khoảng 2007 mm/năm), đặc trưng của khí hậu nhiệt đới gió mùa, lượng mưa dồi dào.
- **Bốc hơi tiềm năng (`pet_mean`):** **4.0 mm/ngày**, mức bốc hơi cao do nhiệt độ và bức xạ mặt trời lớn.
- **Chỉ số khô hạn (`aridity`):** **0.62**, cho thấy khí hậu thuộc loại ẩm đến bán ẩm (sub-humid), không khô cằn.
- **Tính mùa của mưa (`p_seasonality`):** **0.4**, thể hiện sự phân hóa giữa mùa mưa và mùa khô rõ rệt.

### ĐỊA HÌNH:
- **Vĩ độ đại diện (`gauge_lat`):** **16.0° Bắc**, lấy khu vực miền Trung Việt Nam làm vĩ độ tham chiếu cho toàn quốc.
- **Độ cao trung bình (`elev_mean`):** **200 m**, đặc trưng cho địa hình chủ yếu là đồi núi thấp và vùng trung du.
- **Độ dốc trung bình (`slope_mean`):** **15.0 độ**, phản ánh địa hình có độ dốc đáng kể, không phải đồng bằng phẳng.

### ĐẤT:
- **Tỷ lệ cát (`sand_frac`):** **35.0%**
- **Tỷ lệ mùn/phù sa (`silt_frac`):** **25.0%**
- **Tỷ lệ sét (`clay_frac`):** **35.0%**
  - *(Nhận xét: Cấu trúc đất có tỷ lệ sét và cát tương đối cao, điển hình của đất Feralit (đất đỏ vàng) ở vùng nhiệt đới)*
- **Độ xốp của đất (`soil_porosity`):** **0.5** (tương đương 50%), cho thấy đất có khả năng giữ nước và thoát nước ở mức trung bình.

### THỰC VẬT:
- **Tỷ lệ che phủ rừng (`frac_forest`):** **0.42** (tức **42%** diện tích được che phủ bởi rừng, cập nhật theo dữ liệu thực tế của Việt Nam).
- **Chỉ số diện tích lá tối đa (`lai_max`):** **5.5**, cho thấy thảm thực vật rất rậm rạp vào mùa sinh trưởng cao điểm.
- **Tỷ lệ phủ xanh tối đa (`gvf_max`):** **0.8** (tức **80%** bề mặt được che phủ bởi thảm thực vật xanh).

## Lọc ra các gauges có đặc điểm tương đồng với đặc điểm khí hậu ở Việt Nam

### Tính độ tương đồng dựa theo các đặc điểm của lưu vực

In [ ]:
print("Tính điểm tương đồng của các gauges dựa vào các đặc điểm khí hậu Việt Nam")
vietnam_ideal_updated = {
    # Climate
    'p_mean': 5.5,
    'pet_mean': 4.0,
    'aridity': 0.62,
    'p_seasonality': 0.4,

    # Topography
    'gauge_lat': 16.0,
    'elev_mean': 200.0,
    'slope_mean': 15.0,

    # Soil
    'sand_frac': 35.0,
    'silt_frac': 25.0,
    'clay_frac': 35.0,
    'soil_porosity': 0.5,

    # Vegetation (cập nhật theo thực tế VN 42%)
    'frac_forest': 0.42,
    'lai_max': 5.5,
    'gvf_max': 0.8,
}

def calculate_vietnam_similarity_updated(row):
    score = 0
    total_weight = 0

    # Trọng số ưu tiên (đảm bảo được tính quan trọng của mỗi yếu tố làm ảnh hưởng đến lưu lượng dòng chảy)
    weights = {
        # Climate
        'p_mean': 4.5,
        'pet_mean': 3.0,
        'aridity': 4.0,
        'p_seasonality': 3.5,

        # Topography
        'gauge_lat': 3.0,
        'elev_mean': 2.5,
        'slope_mean': 2.0,

        # Soil
        'sand_frac': 2.5,
        'silt_frac': 2.0,
        'clay_frac': 3.5,
        'soil_porosity': 2.0,

        # Vegetation
        'frac_forest': 4.5,
        'lai_max': 3.0,
        'gvf_max': 2.5,
    }

    for param, ideal_value in vietnam_ideal_updated.items():
        if param in row and not pd.isna(row[param]) and param in weights:
            # Tính normalized difference
            if param in ['p_mean', 'pet_mean', 'aridity', 'frac_forest', 'lai_max', 'gvf_max']:
                diff = abs(row[param] - ideal_value) / max(ideal_value, 0.1)
            elif param == 'p_seasonality':
                diff = abs(row[param] - ideal_value) / max(0.5, abs(ideal_value))
            elif param == 'gauge_lat':
                diff = abs(row[param] - ideal_value) / ideal_value
            elif param in ['elev_mean', 'slope_mean']:
                diff = abs(row[param] - ideal_value) / max(ideal_value, 50)
            elif param in ['sand_frac', 'silt_frac', 'clay_frac']:
                diff = abs(row[param] - ideal_value) / max(ideal_value, 10)
            elif param == 'soil_porosity':
                diff = abs(row[param] - ideal_value) / ideal_value

            # Convert to similarity (cải thiện hàm tương đồng)
            similarity = max(0, 1 - min(diff, 1.2))  # Cho phép sai lệch cao hơn một chút
            score += weights[param] * similarity
            total_weight += weights[param]

    return score / total_weight if total_weight > 0 else 0

# Tính điểm tương đồng cho tất cả trạm
df['vietnam_similarity_updated'] = df.apply(calculate_vietnam_similarity_updated, axis=1)

# Sắp xếp theo điểm tương đồng
df_sorted_updated = df.sort_values('vietnam_similarity_updated', ascending=False)

print("\nTOP 20 TRẠM TƯƠNG ĐỒNG CAO NHẤT:")
print("-" * 70)
display_cols_updated = ['gauge_id', 'gauge_name', 'gauge_lat', 'gauge_lon',
                       'p_mean', 'aridity', 'p_seasonality', 'elev_mean',
                       'frac_forest', 'clay_frac', 'vietnam_similarity_updated']
print(df_sorted_updated[display_cols_updated].head(20).to_string(index=False))




Tính điểm tương đồng của các gauges dựa vào các đặc điểm khí hậu Việt Nam

TOP 20 TRẠM TƯƠNG ĐỒNG CAO NHẤT:
----------------------------------------------------------------------
 gauge_id                                    gauge_name  gauge_lat  gauge_lon   p_mean  aridity  p_seasonality  elev_mean  frac_forest  clay_frac  vietnam_similarity_updated
  1638480             CATOCTIN CREEK AT TAYLORSTOWN, VA   39.25455  -77.57638 3.034423 0.843032       0.129989     155.98       0.4659  23.723315                    0.654730
  3357350              PLUM CREEK NEAR BAINBRIDGE, IND.   39.76171  -86.72945 3.220156 0.785652       0.192942     258.64       0.4476  25.530649                    0.645866
  2472000                     LEAF RIVER NR COLLINS, MS   31.70694  -89.40694 4.158402 0.761750      -0.163583     123.24       0.6696  29.054315                    0.644912
  1580000                       DEER CREEK AT ROCKS, MD   39.62997  -76.40331 3.431728 0.736437       0.084633     174.30    

### Export kết quả các gauges có độ tương đồng cao ra file csv

In [ ]:
# Phân loại các gauge dựa vào độ tương đồng
high_similarity_updated = df_sorted_updated[df_sorted_updated['vietnam_similarity_updated'] >= 0.6]
medium_similarity_updated = df_sorted_updated[(df_sorted_updated['vietnam_similarity_updated'] >= 0.5) &
                                             (df_sorted_updated['vietnam_similarity_updated'] < 0.6)]
print(f"\nPHÂN LOẠI THEO ĐIỂM TƯƠNG ĐỒNG:")
print("-" * 50)
print(f"Tương đồng cao (≥0.6): {len(high_similarity_updated)} trạm")
print(f"Tương đồng trung bình (0.5-0.6): {len(medium_similarity_updated)} trạm")
print("="*80)
print("EXPORTING RESULTS")
if len(high_similarity_updated) > 0:
    high_similarity_updated.to_csv('vietnam_similar_gauges_high.csv', index=False)
    print(f"  vietnam_similar_gauges_high.csv ({len(high_similarity_updated)} trạm)")

if len(medium_similarity_updated) > 0:
    medium_similarity_updated.to_csv('vietnam_similar_gauges_medium.csv', index=False)
    print(f"  vietnam_similar_gauges_medium.csv ({len(medium_similarity_updated)} trạm)")


PHÂN LOẠI THEO ĐIỂM TƯƠNG ĐỒNG:
--------------------------------------------------
Tương đồng cao (≥0.6): 30 trạm
Tương đồng trung bình (0.5-0.6): 269 trạm
EXPORTING RESULTS
  vietnam_similar_gauges_high.csv (30 trạm)
  vietnam_similar_gauges_medium.csv (269 trạm)


In [ ]:
plot_gauge_locations(high_similarity_updated)

## Preparing training data - Chuẩn bị dữ liệu huấn luyện.
### Sau khi đã lọc ra các gauge phù hợp, ta tiến hành chuẩn bị data cho training step. Bao gồm các bước sau:


1.   Load ra gauge_ID từ file csv đã tổng hợp được
2.   Tìm file chứa dữ liệu về khí hậu môi trường, dòng chảy đo đạt được
3.   Merge với nhau



In [ ]:
def load_gauge_list(file_path):
    """Load ra gauge_ID có mức độ tương đồng cao"""
    df = pd.read_csv(file_path)
    gauge_ids = df['gauge_id'].astype(str).str.zfill(8).tolist()  # Ensure 8-digit format
    print(f"Loaded {len(gauge_ids)} gauge IDs")
    return gauge_ids

def find_gauge_files(gauge_id, base_path):
    """
    Dựa vào gauge_ID để tìm file chứa dữ liệu khí hậu hoặc môi trường được sử dụng làm đầu vào cho mô hình
    và dữ liệu dòng chảy (đầu ra mong muốn) đo đạt được
    """
    # Search in all HUC regions (01-18)
    for huc in [f"{i:02d}" for i in range(1, 19)]:
        # Check for forcing file
        forcing_file = f"{base_path}/basin_mean_forcing/daymet/{huc}/{gauge_id}_lump_cida_forcing_leap.txt"
        streamflow_file = f"{base_path}/usgs_streamflow/{huc}/{gauge_id}_streamflow_qc.txt"

        if os.path.exists(forcing_file) and os.path.exists(streamflow_file):
            return forcing_file, streamflow_file, huc

    return None, None, None

def read_forcing_data(file_path, start_year=1980, end_year=2014):
    """Đọc file chứa dữ liệu về khí hậu, môi trường"""
    try:
        with open(file_path, 'r') as f:
            lines = f.readlines()

        # 3 dòng đầu của file .txt có chứa metadata về vĩ độ, độ cao và diện tích khu vực khảo sát  (first 3 lines)
        metadata = {
            'latitude': float(lines[0].strip()),
            'elevation': float(lines[1].strip()),
            'area': float(lines[2].strip())
        }

        data_lines = lines[4:]
        data = []
        for line in data_lines:
            parts = line.strip().split()
            if len(parts) >= 10:
                year = int(parts[0])
                if start_year <= year <= end_year:
                    data.append({
                        'year': year,
                        'month': int(parts[1]),
                        'day': int(parts[2]),
                        'hour': int(parts[3]),
                        'dayl': float(parts[4]),      # day length (s)
                        'prcp': float(parts[5]),      # precipitation (mm/day)
                        'srad': float(parts[6]),      # solar radiation (W/m2)
                        'swe': float(parts[7]),       # snow water equivalent (mm)
                        'tmax': float(parts[8]),      # max temperature (C)
                        'tmin': float(parts[9]),      # min temperature (C)
                        'vp': float(parts[10])        # vapor pressure (Pa)
                    })

        df = pd.DataFrame(data)
        if not df.empty:
            df['date'] = pd.to_datetime(df[['year', 'month', 'day']])

        return df, metadata

    except Exception as e:
        print(f"Error reading forcing data from {file_path}: {e}")
        return None, None

def read_streamflow_data(file_path, start_year=1980, end_year=2014):
    """Đọc file chứa dữ liệu dòng chảy"""
    try:
        data = []
        with open(file_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) >= 5:
                    gauge_id = parts[0]
                    year = int(parts[1])
                    month = int(parts[2])
                    day = int(parts[3])
                    flow = float(parts[4])
                    quality = parts[5] if len(parts) > 5 else 'A'

                    if start_year <= year <= end_year:
                        data.append({
                            'gauge_id': gauge_id,
                            'year': year,
                            'month': month,
                            'day': day,
                            'streamflow': flow,
                            'quality': quality
                        })

        df = pd.DataFrame(data)
        if not df.empty:
            df['date'] = pd.to_datetime(df[['year', 'month', 'day']])
            # Filter only high quality data
            df = df[df['quality'].isin(['A', 'A:e'])]  # Approved data

        return df

    except Exception as e:
        print(f"Error reading streamflow data from {file_path}: {e}")
        return None

def process_gauge_data(gauge_id, base_path, start_year=1980, end_year=2014):
    """Merge dữ liệu metforcing và streamflow"""
    print(f"Processing gauge {gauge_id}")

    # Find files
    forcing_file, streamflow_file, huc = find_gauge_files(gauge_id, base_path)

    if not forcing_file or not streamflow_file:
        print(f"Files not found for gauge {gauge_id}")
        return None

    print(f"Found files for gauge {gauge_id} in HUC {huc}")

    # Read data
    forcing_df, metadata = read_forcing_data(forcing_file, start_year, end_year)
    streamflow_df = read_streamflow_data(streamflow_file, start_year, end_year)

    if forcing_df is None or streamflow_df is None:
        print(f"Failed to read data for gauge {gauge_id}")
        return None

    if forcing_df.empty or streamflow_df.empty:
        print(f"Empty data for gauge {gauge_id}")
        return None

    # Merge forcing and streamflow data on date
    merged_df = pd.merge(forcing_df, streamflow_df, on='date', how='inner')

    if merged_df.empty:
        print(f"No matching dates for gauge {gauge_id}")
        return None

    # Add gauge info
    merged_df['gauge_id'] = gauge_id
    merged_df['huc'] = huc

    # Add metadata
    for key, value in metadata.items():
        merged_df[f'basin_{key}'] = value

    print(f"Successfully processed {len(merged_df)} records for gauge {gauge_id}")
    return merged_df

def create_training_dataset(gauge_list_file, base_path, output_file, start_year=1980, end_year=2014):
    """Create complete training dataset for all gauges"""
    print(f"Creating training dataset for period {start_year}-{end_year}")

    # Load gauge list
    gauge_ids = load_gauge_list(gauge_list_file)

    all_data = []
    successful_gauges = []
    failed_gauges = []

    for i, gauge_id in enumerate(gauge_ids, 1):
        print(f"Processing gauge {i}/{len(gauge_ids)}: {gauge_id}")

        gauge_data = process_gauge_data(gauge_id, base_path, start_year, end_year)

        if gauge_data is not None:
            all_data.append(gauge_data)
            successful_gauges.append(gauge_id)
            print(f"✓ Gauge {gauge_id}: {len(gauge_data)} records")
        else:
            failed_gauges.append(gauge_id)
            print(f"✗ Failed to process gauge {gauge_id}")

    if not all_data:
        print("No data was successfully processed!")
        return None

    # Combine all data
    print("Combining all gauge data...")
    final_df = pd.concat(all_data, ignore_index=True)

    # Sort by gauge_id and date
    final_df = final_df.sort_values(['gauge_id', 'date']).reset_index(drop=True)

    # Reorder columns for better readability
    column_order = [
        'gauge_id', 'huc', 'date', 'year_x', 'month_x', 'day_x',
        'basin_latitude', 'basin_elevation', 'basin_area',
        'dayl', 'prcp', 'srad', 'swe', 'tmax', 'tmin', 'vp',
        'streamflow', 'quality'
    ]

    # Only include columns that exist
    available_columns = [col for col in column_order if col in final_df.columns]
    other_columns = [col for col in final_df.columns if col not in available_columns]
    final_columns = available_columns + other_columns

    final_df = final_df[final_columns]

    # Save to CSV
    print(f"Saving dataset to {output_file}")
    final_df.to_csv(output_file, index=False)

    # Print summary
    print("=" * 60)
    print("DATASET CREATION SUMMARY")
    print("=" * 60)
    print(f"Total gauges requested: {len(gauge_ids)}")
    print(f"Successfully processed: {len(successful_gauges)}")
    print(f"Failed to process: {len(failed_gauges)}")
    print(f"Success rate: {len(successful_gauges)/len(gauge_ids)*100:.1f}%")
    print(f"Total records: {len(final_df):,}")
    print(f"Date range: {final_df['date'].min()} to {final_df['date'].max()}")
    print(f"Average records per gauge: {len(final_df)/len(successful_gauges):.0f}")

    if failed_gauges:
        print(f"\nFailed gauges: {failed_gauges}")

    print(f"\nDataset saved to: {output_file}")
    print(f"File size: {os.path.getsize(output_file)/1024/1024:.1f} MB")

    return final_df

if __name__ == "__main__":
    # Configuration
    GAUGE_LIST_FILE = "/content/drive/MyDrive/data/vietnam_similar_gauges_high.csv"
    BASE_PATH = "/content/drive/MyDrive/data/basin_timeseries_v1p2_metForcing_obsFlow/basin_dataset_public_v1p2"
    OUTPUT_FILE = "vietnam_training_dataset_1980_2014.csv"
    START_YEAR = 1980
    END_YEAR = 2014

    # Create dataset
    dataset = create_training_dataset(
        gauge_list_file=GAUGE_LIST_FILE,
        base_path=BASE_PATH,
        output_file=OUTPUT_FILE,
        start_year=START_YEAR,
        end_year=END_YEAR
    )

    if dataset is not None:
        print("Training dataset created successfully!")

        # Display sample data
        print("\nSample data (first 5 rows):")
        print(dataset.head())

        print(f"\nDataset shape: {dataset.shape}")
        print(f"Columns: {list(dataset.columns)}")
    else:
        print("Failed to create training dataset!")

Creating training dataset for period 1980-2014
Loaded 30 gauge IDs
Processing gauge 1/30: 01638480
Processing gauge 01638480
Found files for gauge 01638480 in HUC 02
Successfully processed 12707 records for gauge 01638480
✓ Gauge 01638480: 12707 records
Processing gauge 2/30: 03357350
Processing gauge 03357350
Found files for gauge 03357350 in HUC 05
Successfully processed 12375 records for gauge 03357350
✓ Gauge 03357350: 12375 records
Processing gauge 3/30: 02472000
Processing gauge 02472000
Found files for gauge 02472000 in HUC 03
Successfully processed 12784 records for gauge 02472000
✓ Gauge 02472000: 12784 records
Processing gauge 4/30: 01580000
Processing gauge 01580000
Found files for gauge 01580000 in HUC 02
Successfully processed 12768 records for gauge 01580000
✓ Gauge 01580000: 12768 records
Processing gauge 5/30: 07261000
Processing gauge 07261000
Found files for gauge 07261000 in HUC 11
Successfully processed 12676 records for gauge 07261000
✓ Gauge 07261000: 12676 record

## Training model

## Apply to local (Long Đại River)

[Google Flood Hub](https://sites.research.google/floods/l/17.377209907539214/106.62817225351255/10.773249999999996)
